In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install faiss-cpu --no-cache-dir

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 248.1 MB/s eta 0:00:00a 0:00:01


In [3]:
# ─── Cell 1: Imports & load index ─────────────────────────────────────────────
import faiss
import pickle
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

# 1) Load FAISS index & metadata
index = faiss.read_index("/kaggle/input/indexes/faiss.index")
with open("/kaggle/input/indexes/metadata.pkl", "rb") as f:
    df = pickle.load(f)

# 2) Init your embedder
model = SentenceTransformer("all-MiniLM-L6-v2")


2025-07-30 13:23:36.057755: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753881816.300391      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753881816.378064      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
# ─── Cell 2: Pure dense‐retrieval search function ─────────────────────────────
def search_titles_urls(query: str, k: int = 5) -> pd.DataFrame:
    """
    Returns the top‐k (title, url) for the given query using
    MiniLM embeddings + FAISS inner‐product search.
    """
    qv = model.encode([query]).astype("float32")
    D, I = index.search(qv, k)

    # grab & dedupe
    res = (
        df.iloc[I[0]][["title","url"]]
        .drop_duplicates()
        .reset_index(drop=True)
    )
    return res


In [9]:

from IPython.display import HTML, display

# 1) Run your search as before
query   = "Attention is all you need"
results = search_titles_urls(query, k=10)

# 2) Convert the 'url' column into clickable links
results_html = results.to_html(
    escape=False,
    index=False,
    formatters={
        "url": lambda x: f'<a href="{x}" target="_blank">{x}</a>'
    }
)

# 3) Display as HTML
display(HTML(results_html))


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

title,url
Day 58: Attention Mechanisms — Foundation of Transformer Models | by Adithya Prasad Pandelu | Medium,https://medium.com/@bhatadithya54764118/day-58-attention-mechanisms-foundation-of-transformer-models-c3139d3c0e79
What is an attention mechanism? | IBM,https://www.ibm.com/think/topics/attention-mechanism
"Attention in Deep Learning, your starting point (with code)",https://tungmphung.com/attention-in-deep-learning-your-starting-point-with-code/
Reddit - The heart of the internet,https://www.reddit.com/r/MachineLearning/comments/qidpqx/d_how_to_truly_understand_attention_mechanism_in/
The Mathematics of Attention Mechanisms: Building Self-Attention from First Principles | by Abduldattijo | Generative AI,https://generativeai.pub/the-mathematics-of-attention-mechanisms-building-self-attention-from-first-principles-b9895e9ee491
All you need (to know) about attention | AndoLogs,https://blog.ando.ai/posts/all_you_need_to_know_about_attention/
5 Attention Mechanism Insights Every AI Developer Should Know,https://shelf.io/blog/attention-mechanism/
Understanding the attention mechanism in sequence models,https://www.jeremyjordan.me/attention/
Understanding and Coding the Self-Attention Mechanism of Large Language Models From Scratch,https://sebastianraschka.com/blog/2023/self-attention-from-scratch.html
